In [47]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# 1. Data Loading and Preprocessing
def load_and_preprocess_data(filepath):
    # Load data
    df = pd.read_csv(filepath)
    
    # Preprocessing
    df = df.drop('Customer ID', axis=1)
    df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
    
    # Handle categorical variables
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if df[col].nunique() == 2:
            df[col] = LabelEncoder().fit_transform(df[col])
        else:
            df = pd.get_dummies(df, columns=[col], prefix=col)
    
    return df

# 2. Model Creation
def create_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    return model

# 3. Main Execution
def main():
    try:
        # Step 1: Prepare data
        print("Loading and preprocessing data...")
        df_processed = load_and_preprocess_data('Churn.csv')
        X = df_processed.drop('Churn', axis=1)
        y = df_processed['Churn']
        
        # Step 2: Train-test split
        print("Splitting data into train/test sets...")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y)
        
        # Step 3: Feature scaling
        print("Scaling features...")
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Step 4: Create model
        print("Creating model...")
        input_shape = X_train.shape[1]
        model = create_model(input_shape)
        model.summary()
        
        # Step 5: Train model
        print("Training model...")
        history = model.fit(
            X_train, y_train,
            epochs=10,
            batch_size=32,
            validation_split=0.2,
            verbose=1
        )
        
        # Step 6: Evaluate model
        print("\nEvaluating model...")
        print("Metric names:", model.metrics_names)
        test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test, verbose=0)
        
        print(f"\nTest Accuracy: {test_acc:.4f}")
        print(f"Test Precision: {test_precision:.4f}")
        print(f"Test Recall: {test_recall:.4f}")
        
        # Step 7: Generate predictions
        y_pred = (model.predict(X_test) > 0.5).astype(int)
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))
        print("\nConfusion Matrix:")
        print(confusion_matrix(y_test, y_pred))
        
    except Exception as e:
        print(f"\nError encountered: {str(e)}")
        print("Please check:")
        print("- Data file exists and is in correct format")
        print("- All required columns are present")
        print("- No missing values in critical fields")

if __name__ == "__main__":
    main()

Loading and preprocessing data...
Splitting data into train/test sets...
Scaling features...
Creating model...


C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │       420,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 422,657 (1.61 MB)

 Trainable params: 422,657 (1.61 MB)

 Non-trainable params: 0 (0.00 B)

Training model...
Epoch 1/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.6773 - loss: 0.6011 - precision_1: 0.3411 - recall_1: 0.1996 - val_accuracy: 0.7613 - val_loss: 0.4618 - val_precision_1: 0.6102 - val_recall_1: 0.2441
Epoch 2/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8397 - loss: 0.3559 - precision_1: 0.7601 - recall_1: 0.5810 - val_accuracy: 0.7374 - val_loss: 0.4786 - val_precision_1: 0.4984 - val_recall_1: 0.5322
Epoch 3/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9354 - loss: 0.1686 - precision_1: 0.8882 - recall_1: 0.8746 - val_accuracy: 0.7214 - val_loss: 0.5566 - val_precision_1: 0.4731 - val_recall_1: 0.5661
Epoch 4/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9756 - loss: 0.0700 - precision_1: 0.9476 - recall_1: 0.9600 - val_accuracy: 0.7107 - val_loss: 0.6585 - val_precision_1: 0.4628 - val_recall_1: 0.6542
Epoch 5/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9856 - loss: 0.0414 - precision_1: 0.9723 -